In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Riiid! Answer Correctness Prediction

In [ ]:
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter('ignore')
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.utils import shuffle

import riiideducation

%matplotlib inline
# for heatmap and other plots
colorMap1 = sns.color_palette("RdBu_r")
# for countplot and others plots
colorMap2 = 'Blues_r'

WIDTH=800

In [ ]:
types = {
        'row_id': 'int64', 
        'timestamp': 'int64', 
        'user_id': 'int32', 
        'content_id': 'int16', 
        'content_type_id': 'int8',
        'task_container_id': 'int16', 
        'user_answer': 'int8', 
        'answered_correctly': 'int8', 
        'prior_question_elapsed_time': 'float32', 
        'prior_question_had_explanation': 'boolean'
}

In [ ]:
train = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv', 
    low_memory=False, 
    nrows=10**7, 
    dtype=types
)
questions=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
lectures=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')
test=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')

train.head()

#                  DATA EXPLORATION & EDA

**TRAIN**

**row_id**: (int64) ID code for the row.

**timestamp**: (int64) the time in milliseconds between this user interaction and the first event completion from that user.

**user_id**: (int32) ID code for the user.

**content_id**: (int16) ID code for the user interaction

**content_type_id**: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

**task_container_id**: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.

**user_answer**: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.

**answered_correctly**: (int8) if the user responded correctly. Read -1 as null, for lectures.

**prior_question_elapsed_time**: (float32) The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures
in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.

**prior_question_had_explanation**: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.

In [ ]:
print(f"Train shape: {train.shape}")

In [ ]:

train.head()

**QUESTIONS.CSV**


question_id: foreign key for the train/test content_id column, when the content type is question (0).

bundle_id: code for which questions are served together.

correct_answer: the answer to the question. Can be compared with the train user_answer column to check if the user was right.

part: top level category code for the question.

tags: one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.

In [ ]:
questions.head()

In [ ]:
questions.describe().style.background_gradient(cmap='Blues')

In [ ]:
print(questions.isnull().sum() / len(questions))

In [ ]:





questions['tag'] = questions['tags'].str.split(' ')
questions = questions.explode('tag')
questions = pd.merge(
    questions, 
    questions.groupby('question_id')['tag'].count().reset_index(), 
    on='question_id'
    )

questions = questions.drop(['tag_x'], axis=1)

questions.columns = [
    'question_id', 
    'bundle_id', 
    'correct_answer', 
    'part', 
    'tags', 
    'tags_number'
]
questions = questions.drop_duplicates()

In [ ]:
ds = questions['correct_answer'].value_counts().reset_index()

ds.columns = [
    'correct_answer', 
    'number_of_answers'
]

ds['correct_answer'] = ds['correct_answer'].astype(str) + '-'
ds = ds.sort_values(['number_of_answers'])

fig = px.bar(
    ds, 
    x='number_of_answers', 
    y='correct_answer', 
    orientation='h', 
    title='Number of correct answers per group', 
    width=WIDTH,
    height=300
)
fig.show()

In [ ]:
ds = questions['part'].value_counts().reset_index()

ds.columns = [
    'part', 
    'count'
]

ds['part'] = ds['part'].astype(str) + '-'
ds = ds.sort_values(['count'])

fig = px.bar(
    ds, 
    x='count', 
    y='part', 
    orientation='h', 
    title='Parts distribution',
    width=WIDTH,
    height=400
)
fig.show()

In [ ]:
ds = questions['tags_number'].value_counts().reset_index()

ds.columns = [
    'tags_number', 
    'count'
]

ds['tags_number'] = ds['tags_number'].astype(str) + '-'
ds = ds.sort_values(['tags_number'])

fig = px.bar(
    ds, 
    x='count', 
    y='tags_number', 
    orientation='h', 
    title='Number tags distribution', 
    width=WIDTH,
    height=400 
    )

fig.show()

In [ ]:


check = questions['tags'].str.split(' ').explode('tags').reset_index()
check = check['tags'].value_counts().reset_index()

check.columns = [
    'tag', 
    'count'
]

check['tag'] = check['tag'].astype(str) + '-'
check = check.sort_values(['count']).tail(40)

fig = px.bar(
    check, 
    x='count', 
    y='tag', 
    orientation='h', 
     title='Top 40 most useful tags', 
    width=WIDTH,
    height=900 
)

fig.show()


LECTURES.CSV

Metadata for the lectures watched by users as they progress in their education.

**lecture_id**: foreign key for the train/test content_id column, when the content type is lecture (1).

**part**: top level category code for the lecture.

**tag**: one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.

**type_of**: brief description of the core purpose of the lecture

In [ ]:
lectures.head(10)

In [ ]:
print('Part of missing values for every column')
print(lectures.isnull().sum() / len(lectures))

In [ ]:
lectures['type_of'].value_counts()

**Test.csv**

In [ ]:
test.head()

# FEATURE ENGINEERING

In [ ]:
train.head()

In [ ]:
features_df = train.iloc[:int(9/10 * len(train))]
train = train.iloc[int(9/10 * len(train)):]

In [ ]:
train_questions_only_df = features_df[features_df['answered_correctly']!=-1]
grouped_by_user_df = train_questions_only_df.groupby('user_id')
features_full_df = features_df

user_answers_df = grouped_by_user_df.agg(
    {
        'answered_correctly': [
            'mean', 
            'count', 
            'sum',
            'std', 
            'median', 
            'skew']}
).copy()

user_answers_df.columns = [
    'mean_user_accuracy',
    'questions_answered',
    'user_correct_answers',
    'std_user_accuracy', 
    'median_user_accuracy', 
    'skew_user_accuracy'
]

In [ ]:
grouped_by_content_df = train_questions_only_df.groupby('content_id')
content_answers_df = grouped_by_content_df.agg(
    {
        'answered_correctly': [
            'mean', 
            'count', 
            'sum',
            'std', 
            'median', 
            'skew'
        ]
    }
).copy()

content_answers_df.columns = [
    'mean_accuracy', 
    'question_asked', 
    'content_positive_answers',
    'std_accuracy', 
    'median_accuracy', 
    'skew_accuracy'
]
content_answers_df['content_positive_answers'] = content_answers_df['content_positive_answers'].astype(int)

content_answers_df

In [ ]:
grouped_by_container_df = features_full_df.groupby('task_container_id')
container_df = grouped_by_container_df.agg(
    {'answered_correctly': [
            'mean', 
            'count'],
    'content_type_id':['sum']
    }).copy()
#lectures = train_full_df.groupby('task_container_id')["content_type_id"].sum()
container_df.columns = [
    'mean_container', 
    'questions_container',
    'lecs_in_container']
container_df["lecs_in_container"] = container_df["lecs_in_container"].astype(int)
container_df["lecs_to_qs"] = container_df["lecs_in_container"]/container_df["questions_container"]


In [ ]:
del features_df
del grouped_by_user_df
del grouped_by_content_df
del grouped_by_container_df

In [ ]:
features = [
    'mean_user_accuracy', 
    'questions_answered',
    'user_correct_answers',
    'std_user_accuracy', 
    'median_user_accuracy',
    'skew_user_accuracy',
    'mean_accuracy', 
    'question_asked',
    'std_accuracy', 
    'content_positive_answers',
    'median_accuracy',
    'prior_question_elapsed_time', 
    'prior_question_had_explanation',
    'skew_accuracy' ,  
    'mean_container', 
    'questions_container',
    'lecs_in_container',
    'lecs_to_qs']
target = 'answered_correctly'

In [ ]:
target

In [ ]:
train = train[train[target] != -1]

In [ ]:
train = train.merge(user_answers_df, how='left', on='user_id')
train = train.merge(content_answers_df, how='left', on='content_id')
train = train.merge(container_df, how='left', on='task_container_id')

In [ ]:
train.head()


In [ ]:

train['prior_question_had_explanation'] = train['prior_question_had_explanation'].fillna(value=False).astype(bool)
df = train.fillna(train.mean())

In [ ]:
col_to_drop = set(train.columns.values.tolist()).difference(features + [target])
for col in col_to_drop:
    del df[col]

In [ ]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(df.mean())

In [ ]:
df.head()


# Modeling

In [ ]:
train_df, test_df, y_train, y_test = train_test_split(df[features], df[target], random_state=3, test_size=0.25)

# Random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 100, max_depth=5, criterion = "entropy")
model.fit(train_df, y_train)
print('Forest ROC-AUC score: ', roc_auc_score(y_test.values, model.predict_proba(test_df)[:, 1]))


Submission preparation

In [ ]:
env = riiideducation.make_env()

In [ ]:
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    # merge
    test_df = test_df.merge(user_answers_df, on = "user_id", how = "left")
    test_df = test_df.merge(content_answers_df, on = "content_id", how = "left")
    test_df = test_df.merge(container_df, how='left', on='task_container_id')
    
    # type transformation
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(value=False).astype(bool)
    test_df.fillna(df.mean(), inplace = True)
    
    # preds
    test_df['answered_correctly'] = model.predict_proba(test_df[features])[:, 1]
    cols_to_submission = ['row_id', 'answered_correctly', 'group_num']
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])